# Classification supervisée avec les modules KNeighborsClassifier et RandomForestClassifier de SCIKIT-Learn

In [76]:
#chargement des bibliothèques
import pandas as pd

In [184]:
#chargement des bibliothèque de machine learning
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [207]:
#chargement des données 
file_path = "C:/Users/Patrick Menan/Desktop/TP1/classification_supervisée/data_set.csv"
df = pd.read_csv(file_path)

In [208]:
df.head()

,X,Y,Classe
0,-0.697580,0.68494,0
1,-0.478690,0.63377,1
2,0.057028,0.91886,0
3,-0.593890,0.49488,0
4,0.229840,-0.41155,1


In [209]:
y = df.loc[:, 'Classe']
X = df.drop('Classe', axis = 'columns')
#y = df.loc[:, 'categorical_classe']
#X = df.drop('categorical_classe', axis = 'columns')

In [210]:
#separation apprentissage(train) et validation(valid)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.8)

In [211]:
#creation du model de classification par forêt aleatoire
model_forest_classifier = RandomForestClassifier(n_estimators=1000, random_state=1)

In [212]:
#entrainement du model
model_forest_classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000, random_state=1)

In [213]:
#creation du model de classification par la methode des k-ppv (5-ppv dans notre cas ici)
model_neighbors_classifier = KNeighborsClassifier()

In [214]:
#entrainement du modèle
model_neighbors_classifier.fit(X_train, y_train)

KNeighborsClassifier()

In [216]:
#evaluation de la performance forêt du modèle sur les données d'apprantissage
model_forest_classifier.score(X_train, y_train)

1.0

In [217]:
#evaluation de la performance du modèle k-ppv les données d'apprentissage
model_neighbors_classifier.score(X_train, y_train)

0.8297872340425532

In [221]:
#prediction avec le modèle Forêt
predict_forest=model_forest_classifier.predict(X_valid)
predict_forest

array([0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 0], dtype=int64)

In [223]:
#prediction avec le modèle K-ppv
predict_neighbors=model_neighbors_classifier.predict(X_valid)
predict_neighbors

array([0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 0], dtype=int64)

In [224]:
#fonction de prédiction d'un nouvel individu
def class_indiv(model, x, y):
    s = pd.DataFrame({'x' : [x], 'y': [y]})
    print(model.predict_proba(s))
    return model.predict(s)

""" 
def class_indiv(model, x, y):
    s = np.array([x, y]).reshape(1,2)
    print(model.predict_proba(s))
    return model.predict(s)
"""


' \ndef class_indiv(model, x, y):\n    s = np.array([x, y]).reshape(1,2)\n    print(model.predict_proba(s))\n    return model.predict(s)\n'

In [234]:
#prédiction d'un nouvel individu (x = 0.23, y = 0.2956) avec le model de classification par forêt aléatoire
class_indiv(model_forest_classifier, 0.23, 0.2956)

[[0.048 0.952]]


array([1], dtype=int64)

In [235]:
#prédiction d'un nouvel individu avec le modèle de classification par k-ppv, ayant les mêmes caratérisques que précédements
class_indiv(model_neighbors_classifier, 0.23, 0.2956)

[[0.2 0.8]]


array([1], dtype=int64)